Import Database

In [13]:
#cond = {"sit": 1, "stand": 2, "exercize": 3, "supine": 4, "tripod", 5}
freq = 200 #Device sampling frequency in Hz
import pandas as pd
df = pd.read_csv('uoftdb.csv')
df.head()
# Uoft ECG Database is organazie in cell arrays as follows:
# signals[#subject][#week][#cond]

,1_1_sit,2_1_sit,3_1_sit,4_1_sit,5_1_sit,6_1_sit,7_1_sit,8_1_sit,9_1_sit,10_1_sit,...,1017_6_sit,1017_6_stand,1018_5_supine,1018_5_tripod,1018_6_sit,1018_6_stand,1019_6_sit,1019_6_stand,1020_6_sit,1020_6_stand
0,0.994,1.207,0.937,0.823,1.078,1.501,1.131,1.104,0.999,1.071,...,0.880508,0.939484,1.000366,0.896378,0.963364,0.573120,0.911636,0.857086,0.837173,0.958405
1,1.008,1.328,0.735,0.712,1.018,2.804,1.016,1.057,0.995,1.134,...,0.881805,0.950775,0.982590,0.907822,0.960846,0.638428,0.888824,0.855789,0.820923,0.994720
2,1.235,1.157,1.293,1.841,1.053,0.314,1.124,1.048,1.224,1.080,...,0.884323,0.974655,0.972443,0.911636,0.948257,0.716171,0.867233,0.863419,0.822144,1.007156
3,1.060,1.159,0.910,1.133,1.095,0.586,1.198,1.103,1.087,1.193,...,0.869217,1.063690,0.980072,0.920486,0.943298,0.762634,0.852051,0.846939,0.830917,0.984726
4,0.985,1.308,0.588,0.609,1.036,3.259,1.031,1.166,0.989,1.627,...,0.869217,1.028595,0.990219,0.938263,0.936966,0.721207,0.858383,0.830460,0.828400,0.970917


Low Pass Filter

In [14]:
from scipy.signal import butter,filtfilt

# Filter requirements.
T = 5.0         # Sample Period
fs = 30.0       # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples

def butter_bandpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='bandpass', analog=False)
    y = filtfilt(b, a, data)
    return y

Filter Bonus

In [15]:
import numpy as np
from scipy.signal import butter,filtfilt
import plotly.graph_objects as go

# Filter requirements.
T = 5.0         # Sample Period
fs = 30.0       # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples

# sin wave
t = np.linspace(0.0, 5.0, num= 150) # 150 samples evenly spaced
sig = np.sin(1.2*2*np.pi*t)

# Lets add some noise
noise = 1.5*np.cos(9*2*np.pi*t) + 0.5*np.sin(12.0*2*np.pi*t)
data = sig + noise

def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = fs /2
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y
    # Filter the data, and plot both the original and filtered signals.
y = butter_lowpass_filter(data, cutoff, fs, order)
fig = go.Figure()
fig.add_trace(go.Scatter(
            y = data,
            line =  dict(shape =  'spline' ),
            name = 'signal with noise'
            ))
fig.add_trace(go.Scatter(
            y = y,
            line =  dict(shape =  'spline' ),
            name = 'filtered signal'
            ))
fig.add_trace(go.Scatter(
            y = sig,
            line =  dict(shape =  'spline' ),
            name = 'original signal'
            ))
fig.show()

In [31]:
# Filter requirements.
T = 5.0         # Sample Period
fs = 200.0       # sample rate, Hz
cutoff = [5, 40]      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 6       
n = int(T * fs) # total number of samples
data = df['1_1_sit'].to_numpy()
data = data[4000:5000]
def butter_bandpass_filter(data, cutoff, fs, order):
    nyq = fs/2
    normal_cutoff = [f/nyq for f in cutoff]
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='bandpass', analog=False)
    y = filtfilt(b, a, data)
    return y

filttered = butter_bandpass_filter(data, cutoff, fs, order)

filt_low = butter_lowpass_filter(data, 40, fs, 6)
fig = go.Figure()
fig.add_trace(go.Scatter(
            y = data,
            line =  dict(shape =  'spline' ),
            name = 'signal with noise'
            ))
fig.add_trace(go.Scatter(
            y = filttered,
            line =  dict(shape =  'spline' ),
            name = 'filtered signal'
            ))
fig.add_trace(go.Scatter(
            y = filt_low,
            line =  dict(shape =  'spline' ),
            name = 'filtered low signal'
            ))
for cutoffi in [0.5]:
    for orderi in [6]:
        fig.add_trace(go.Scatter(
                y = butter_bandpass_filter(data, [cutoffi, 40], fs, orderi),
                line =  dict(shape =  'spline' ),
                name = 'filtered low signal' + str(orderi) + str(cutoffi)
                ))
fig.show()